<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/giswqs/geemap/tree/master/examples/notebooks/ipyleaflet_draw_control.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/notebooks/ipyleaflet_draw_control.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/ipyleaflet_draw_control.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Using ipyleaflet drawing tools to interact with Earth Engine data

* [Video demo](https://i.imgur.com/SzauIsZ.gifv)

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

In [15]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])

In [16]:
import ee
import geemap

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [17]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map.add_basemap('HYBRID') # Add Google Satellite
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

### Draw any shapes on the map above using the Draw Control.

In [18]:
# Retrieves ee.Feature() of the last drawing object.
Map.draw_last_feature

In [19]:
# Retrieves ee.Feature() of all drawing objects.
Map.draw_features

[]

## Clipping Earth Engine Image layer with the Draw Control 

In [20]:
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from geemap import geojson_to_ee

Map = geemap.Map(center=[40,-100], zoom=4)
Map.add_basemap('HYBRID') # Add 

# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Get basemap layers
base_layers = Map.layers

# Add Earth Engine DEM to map
Map.addLayer(image, vis_params, 'SRTM DEM')

# An empty list for storing drawing geometries
feat_list = []

# Get the DrawControl
dc = Map.draw_control

# Handle draw events
def handle_draw(self, action, geo_json):

    geom = geojson_to_ee(geo_json, False)
    feature = ee.Feature(geom)
    feat_list.append(feature)
    collection = ee.FeatureCollection(feat_list)
    clip_image = image.clipToCollection(collection)
    
    Map.layers = base_layers[:3]
    Map.addLayer(clip_image, vis_params, 'SRTM DEM')
#     Map.addLayer(ee.Image().paint(collection, 0, 2), {'palette': 'red'}, 'EE Geometry')
    Map.addLayer(collection, {}, 'Drawing Features')

dc.on_draw(handle_draw)

# # Add a button to the map
# button = widgets.Button(description="Clear drawings")
# btn_control = WidgetControl(widget=button, position='bottomright')
# Map.add_control(btn_control)

# # Handle click event
# def on_button_clicked(b):
#     dc.clear()

# button.on_click(on_button_clicked)

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [21]:
# Print out the geojson of the last drawing object
dc.last_draw

{'type': 'Feature', 'geometry': None}